In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 🏠 House Price Prediction - Linear Regression\n",
    "## Level 1 - Task 2: Build a Simple Linear Regression Model\n",
    "\n",
    "### 📋 Task Description\n",
    "Build a linear regression model to predict house prices using the Boston Housing dataset. This notebook covers everything from data preprocessing to model evaluation.\n",
    "\n",
    "### 🎯 Objectives\n",
    "- Load and explore the raw dataset\n",
    "- Preprocess data (handle missing values, encode categories, scale features)\n",
    "- Split data into training and testing sets\n",
    "- Train a linear regression model\n",
    "- Interpret model coefficients\n",
    "- Evaluate model using R-squared and MSE\n",
    "- Visualize results"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Import required libraries\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from sklearn.model_selection import train_test_split\n",
    "from sklearn.preprocessing import StandardScaler, LabelEncoder\n",
    "from sklearn.linear_model import LinearRegression\n",
    "from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error\n",
    "import warnings\n",
    "warnings.filterwarnings('ignore')\n",
    "\n",
    "# Set style for better visualizations\n",
    "plt.style.use('seaborn-v0_8-darkgrid')\n",
    "sns.set_palette(\"husl\")\n",
    "%matplotlib inline\n",
    "\n",
    "print(\"✅ Libraries imported successfully!\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. 📂 Load and Explore the Dataset"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Define column names (based on Boston Housing dataset)\n",
    "column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', \n",
    "                'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']\n",
    "\n",
    "# Load the dataset\n",
    "df = pd.read_csv('../data/house_prediction_raw.csv', header=None, delim_whitespace=True)\n",
    "df.columns = column_names\n",
    "\n",
    "print(f\"✅ Dataset loaded successfully!\")\n",
    "print(f\"📊 Dataset shape: {df.shape[0]} rows × {df.shape[1]} columns\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Display first few rows\n",
    "print(\"📋 First 5 rows of the dataset:\")\n",
    "df.head()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Dataset information\n",
    "print(\"📋 Dataset Info:\")\n",
    "df.info()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Statistical summary\n",
    "print(\"📊 Statistical Summary:\")\n",
    "df.describe().round(2)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Check for missing values\n",
    "missing_values = df.isnull().sum()\n",
    "print(\"🔍 Missing values per column:\")\n",
    "print(missing_values[missing_values > 0] if any(missing_values > 0) else \"✅ No missing values found!\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. 🔍 Exploratory Data Analysis (EDA)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Target variable distribution\n",
    "plt.figure(figsize=(12, 5))\n",
    "\n",
    "plt.subplot(1, 2, 1)\n",
    "sns.histplot(df['MEDV'], kde=True, color='skyblue')\n",
    "plt.axvline(df['MEDV'].mean(), color='red', linestyle='--', label=f\"Mean: ${df['MEDV'].mean():.2f}K\")\n",
    "plt.axvline(df['MEDV'].median(), color='green', linestyle='--', label=f\"Median: ${df['MEDV'].median():.2f}K\")\n",
    "plt.xlabel('House Price ($1000s)')\n",
    "plt.ylabel('Frequency')\n",
    "plt.title('Distribution of House Prices', fontsize=14, fontweight='bold')\n",
    "plt.legend()\n",
    "\n",
    "plt.subplot(1, 2, 2)\n",
    "sns.boxplot(y=df['MEDV'], color='skyblue')\n",
    "plt.ylabel('House Price ($1000s)')\n",
    "plt.title('Box Plot of House Prices', fontsize=14, fontweight='bold')\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Correlation matrix\n",
    "plt.figure(figsize=(12, 10))\n",
    "correlation_matrix = df.corr()\n",
    "sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, \n",
    "            square=True, linewidths=1, cbar_kws={\"shrink\": 0.8})\n",
    "plt.title('Correlation Matrix of Features', fontsize=16, fontweight='bold')\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Top correlations with target variable\n",
    "target_corr = correlation_matrix['MEDV'].sort_values(ascending=False)\n",
    "print(\"📊 Top features correlated with house price:\")\n",
    "print(target_corr)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Visualize top correlations\n",
    "plt.figure(figsize=(12, 6))\n",
    "top_features = target_corr[1:6].index.tolist()  # Top 5 features (excluding MEDV itself)\n",
    "\n",
    "for i, feature in enumerate(top_features, 1):\n",
    "    plt.subplot(2, 3, i)\n",
    "    plt.scatter(df[feature], df['MEDV'], alpha=0.5)\n",
    "    plt.xlabel(feature)\n",
    "    plt.ylabel('MEDV')\n",
    "    plt.title(f'{feature} vs MEDV (r={correlation_matrix.loc[feature, \"MEDV\"]:.2f})')\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. 🛠️ Data Preprocessing"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Create a copy for preprocessing\n",
    "df_processed = df.copy()\n",
    "\n",
    "print(\"📋 Categorical Variables:\")\n",
    "print(f\"CHAS (Charles River): {df_processed['CHAS'].unique()} values\")\n",
    "print(f\"RAD (Highway Access): {sorted(df_processed['RAD'].unique())}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# One-hot encode RAD (categorical variable)\n",
    "print(\"\\n🔄 Applying one-hot encoding to RAD...\")\n",
    "rad_dummies = pd.get_dummies(df_processed['RAD'], prefix='RAD', drop_first=False)\n",
    "df_encoded = pd.concat([df_processed.drop('RAD', axis=1), rad_dummies], axis=1)\n",
    "\n",
    "print(f\"Shape before encoding: {df_processed.shape}\")\n",
    "print(f\"Shape after encoding: {df_encoded.shape}\")\n",
    "print(f\"New columns created: {list(rad_dummies.columns)}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Separate features and target\n",
    "X = df_encoded.drop('MEDV', axis=1)\n",
    "y = df_encoded['MEDV']\n",
    "\n",
    "print(f\"Features shape: {X.shape}\")\n",
    "print(f\"Target shape: {y.shape}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Split data into training and testing sets\n",
    "X_train, X_test, y_train, y_test = train_test_split(\n",
    "    X, y, test_size=0.2, random_state=42\n",
    ")\n",
    "\n",
    "print(f\"✅ Training set: {X_train.shape[0]} samples\")\n",
    "print(f\"✅ Testing set: {X_test.shape[0]} samples\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Scale features using StandardScaler\n",
    "scaler = StandardScaler()\n",
    "\n",
    "# Fit on training data only, then transform both sets\n",
    "X_train_scaled = scaler.fit_transform(X_train)\n",
    "X_test_scaled = scaler.transform(X_test)\n",
    "\n",
    "# Convert back to DataFrames for easier handling\n",
    "X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)\n",
    "X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)\n",
    "\n",
    "print(\"✅ Features scaled using StandardScaler\")\n",
    "print(f\"\\nTraining set - mean (should be ~0):\")\n",
    "print(X_train_scaled.mean().round(2).head())\n",
    "print(f\"\\nTraining set - std (should be ~1):\")\n",
    "print(X_train_scaled.std().round(2).head())"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Visualize effect of scaling\n",
    "fig, axes = plt.subplots(2, 2, figsize=(12, 10))\n",
    "\n",
    "# Before scaling\n",
    "axes[0, 0].hist(X_train.iloc[:, 0], bins=30, alpha=0.7, color='skyblue')\n",
    "axes[0, 0].set_title(f'{X_train.columns[0]} - Before Scaling')\n",
    "axes[0, 0].set_xlabel('Value')\n",
    "\n",
    "axes[0, 1].hist(X_train.iloc[:, 1], bins=30, alpha=0.7, color='lightcoral')\n",
    "axes[0, 1].set_title(f'{X_train.columns[1]} - Before Scaling')\n",
    "axes[0, 1].set_xlabel('Value')\n",
    "\n",
    "# After scaling\n",
    "axes[1, 0].hist(X_train_scaled.iloc[:, 0], bins=30, alpha=0.7, color='skyblue')\n",
    "axes[1, 0].set_title(f'{X_train.columns[0]} - After Scaling')\n",
    "axes[1, 0].set_xlabel('Standardized Value')\n",
    "\n",
    "axes[1, 1].hist(X_train_scaled.iloc[:, 1], bins=30, alpha=0.7, color='lightcoral')\n",
    "axes[1, 1].set_title(f'{X_train.columns[1]} - After Scaling')\n",
    "axes[1, 1].set_xlabel('Standardized Value')\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. 🤖 Train Linear Regression Model"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Initialize and train the model\n",
    "model = LinearRegression()\n",
    "model.fit(X_train_scaled, y_train)\n",
    "\n",
    "print(\"✅ Model training complete!\")\n",
    "print(f\"\\n📊 Model Parameters:\")\n",
    "print(f\"   Intercept (bias): ${model.intercept_:.4f}K\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Make predictions\n",
    "y_train_pred = model.predict(X_train_scaled)\n",
    "y_test_pred = model.predict(X_test_scaled)\n",
    "\n",
    "print(\"✅ Predictions generated for both training and test sets\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. 📈 Interpret Model Coefficients"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Create coefficients dataframe\n",
    "coefficients = pd.DataFrame({\n",
    "    'Feature': X_train.columns,\n",
    "    'Coefficient': model.coef_\n",
    "})\n",
    "coefficients['Abs_Coefficient'] = np.abs(coefficients['Coefficient'])\n",
    "coefficients = coefficients.sort_values('Abs_Coefficient', ascending=False)\n",
    "\n",
    "print(\"🔍 Top 10 Most Influential Features:\")\n",
    "coefficients.head(10).style.background_gradient(cmap='coolwarm', subset=['Coefficient'])"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Visualize coefficients\n",
    "plt.figure(figsize=(12, 8))\n",
    "top_features = coefficients.head(15)\n",
    "colors = ['#2ecc71' if x > 0 else '#e74c3c' for x in top_features['Coefficient']]\n",
    "\n",
    "plt.barh(range(len(top_features)), top_features['Coefficient'], color=colors)\n",
    "plt.yticks(range(len(top_features)), top_features['Feature'])\n",
    "plt.xlabel('Coefficient Value (Impact on House Price in $1000s)', fontsize=12)\n",
    "plt.title('Top 15 Feature Coefficients - Impact on House Price', fontsize=14, fontweight='bold')\n",
    "plt.axvline(x=0, color='black', linestyle='-', linewidth=0.5)\n",
    "plt.grid(axis='x', alpha=0.3)\n",
    "\n",
    "# Add value labels\n",
    "for i, v in enumerate(top_features['Coefficient']):\n",
    "    plt.text(v + (0.5 if v > 0 else -2.5), i, f'{v:.2f}', va='center', fontsize=9)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Interpretation\n",
    "print(\"📌 Feature Interpretation:\")\n",
    "print(\"   • Positive coefficient → Feature increases house price\")\n",
    "print(\"   • Negative coefficient → Feature decreases house price\")\n",
    "print(\"\\n📊 Top 3 Positive Impact Features:\")\n",
    "for _, row in coefficients[coefficients['Coefficient'] > 0].head(3).iterrows():\n",
    "    print(f\"   • 1 unit increase in {row['Feature']} INCREASES price by ${row['Coefficient']:.2f}K\")\n",
    "\n",
    "print(\"\\n📊 Top 3 Negative Impact Features:\")\n",
    "for _, row in coefficients[coefficients['Coefficient'] < 0].head(3).iterrows():\n",
    "    print(f\"   • 1 unit increase in {row['Feature']} DECREASES price by ${abs(row['Coefficient']):.2f}K\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. 📊 Evaluate Model Performance"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Calculate metrics\n",
    "train_r2 = r2_score(y_train, y_train_pred)\n",
    "test_r2 = r2_score(y_test, y_test_pred)\n",
    "train_mse = mean_squared_error(y_train, y_train_pred)\n",
    "test_mse = mean_squared_error(y_test, y_test_pred)\n",
    "train_rmse = np.sqrt(train_mse)\n",
    "test_rmse = np.sqrt(test_mse)\n",
    "train_mae = mean_absolute_error(y_train, y_train_pred)\n",
    "test_mae = mean_absolute_error(y_test, y_test_pred)\n",
    "\n",
    "# Display metrics\n",
    "metrics_df = pd.DataFrame({\n",
    "    'Metric': ['R² Score', 'MSE', 'RMSE', 'MAE'],\n",
    "    'Training': [f'{train_r2:.4f}', f'{train_mse:.2f}', f'${train_rmse:.2f}K', f'${train_mae:.2f}K'],\n",
    "    'Test': [f'{test_r2:.4f}', f'{test_mse:.2f}', f'${test_rmse:.2f}K', f'${test_mae:.2f}K']\n",
    "})\n",
    "\n",
    "print(\"📊 Model Performance Metrics:\")\n",
    "metrics_df.style.set_properties(**{'text-align': 'center'})"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Interpretation\n",
    "print(f\"\\n📌 Model Interpretation:\")\n",
    "print(f\"   • R² Score: The model explains {test_r2*100:.1f}% of the variance in house prices\")\n",
    "print(f\"   • RMSE: On average, predictions are off by ${test_rmse:.2f}K\")\n",
    "print(f\"   • MAE: Average absolute error is ${test_mae:.2f}K\")\n",
    "print(f\"   • 95% of predictions are within ±${test_rmse*1.96:.2f}K\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7. 🎨 Visualize Results"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# 7.1 Actual vs Predicted Plot\n",
    "fig, axes = plt.subplots(1, 2, figsize=(15, 6))\n",
    "\n",
    "# Training set\n",
    "axes[0].scatter(y_train, y_train_pred, alpha=0.6, color='#3498db', edgecolors='black', linewidth=0.5)\n",
    "axes[0].plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], 'r--', linewidth=2)\n",
    "axes[0].set_xlabel('Actual House Price ($1000s)', fontsize=12)\n",
    "axes[0].set_ylabel('Predicted House Price ($1000s)', fontsize=12)\n",
    "axes[0].set_title(f'Training Set (R² = {train_r2:.4f})', fontsize=14, fontweight='bold')\n",
    "axes[0].grid(True, alpha=0.3)\n",
    "\n",
    "# Test set\n",
    "axes[1].scatter(y_test, y_test_pred, alpha=0.6, color='#e67e22', edgecolors='black', linewidth=0.5)\n",
    "axes[1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', linewidth=2)\n",
    "axes[1].set_xlabel('Actual House Price ($1000s)', fontsize=12)\n",
    "axes[1].set_ylabel('Predicted House Price ($1000s)', fontsize=12)\n",
    "axes[1].set_title(f'Test Set (R² = {test_r2:.4f})', fontsize=14, fontweight='bold')\n",
    "axes[1].grid(True, alpha=0.3)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# 7.2 Residuals Plot\n",
    "fig, axes = plt.subplots(1, 2, figsize=(15, 6))\n",
    "\n",
    "residuals_train = y_train - y_train_pred\n",
    "residuals_test = y_test - y_test_pred\n",
    "\n",
    "# Training residuals\n",
    "axes[0].scatter(y_train_pred, residuals_train, alpha=0.6, color='#3498db', edgecolors='black', linewidth=0.5)\n",
    "axes[0].axhline(y=0, color='r', linestyle='--', linewidth=2)\n",
    "axes[0].set_xlabel('Predicted House Price ($1000s)', fontsize=12)\n",
    "axes[0].set_ylabel('Residuals ($1000s)', fontsize=12)\n",
    "axes[0].set_title('Training Set Residuals', fontsize=14, fontweight='bold')\n",
    "axes[0].grid(True, alpha=0.3)\n",
    "\n",
    "# Test residuals\n",
    "axes[1].scatter(y_test_pred, residuals_test, alpha=0.6, color='#e67e22', edgecolors='black', linewidth=0.5)\n",
    "axes[1].axhline(y=0, color='r', linestyle='--', linewidth=2)\n",
    "axes[1].set_xlabel('Predicted House Price ($1000s)', fontsize=12)\n",
    "axes[1].set_ylabel('Residuals ($1000s)', fontsize=12)\n",
    "axes[1].set_title('Test Set Residuals', fontsize=14, fontweight='bold')\n",
    "axes[1].grid(True, alpha=0.3)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# 7.3 Residual Distribution\n",
    "fig, axes = plt.subplots(1, 2, figsize=(15, 6))\n",
    "\n",
    "# Training residuals distribution\n",
    "sns.histplot(residuals_train, kde=True, color='#3498db', bins=30, ax=axes[0])\n",
    "axes[0].axvline(x=0, color='r', linestyle='--', linewidth=2)\n",
    "axes[0].axvline(x=residuals_train.mean(), color='green', linestyle='--', \n",
    "                label=f'Mean: {residuals_train.mean():.2f}')\n",
    "axes[0].set_xlabel('Residuals ($1000s)', fontsize=12)\n",
    "axes[0].set_title('Training Set - Residual Distribution', fontsize=14, fontweight='bold')\n",
    "axes[0].legend()\n",
    "\n",
    "# Test residuals distribution\n",
    "sns.histplot(residuals_test, kde=True, color='#e67e22', bins=30, ax=axes[1])\n",
    "axes[1].axvline(x=0, color='r', linestyle='--', linewidth=2)\n",
    "axes[1].axvline(x=residuals_test.mean(), color='green', linestyle='--',\n",
    "                label=f'Mean: {residuals_test.mean():.2f}')\n",
    "axes[1].set_xlabel('Residuals ($1000s)', fontsize=12)\n",
    "axes[1].set_title('Test Set - Residual Distribution', fontsize=14, fontweight='bold')\n",
    "axes[1].legend()\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 8. 📝 Model Equation"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Generate model equation\n",
    "print(\"📝 Linear Regression Model Equation:\")\n",
    "print(\"=\" * 60)\n",
    "print(f\"House Price = {model.intercept_:.2f}\")\n",
    "\n",
    "# Add top 5 features\n",
    "top_5 = coefficients.head(5)\n",
    "for _, row in top_5.iterrows():\n",
    "    print(f\"               {row['Coefficient']:+.2f} × {row['Feature']}\")\n",
    "print(\"               + ... (other features)\")\n",
    "print(\"=\" * 60)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 9. 💾 Save Processed Data and Model"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Create output directories\n",
    "import os\n",
    "os.makedirs('../output/processed', exist_ok=True)\n",
    "os.makedirs('../output/models', exist_ok=True)\n",
    "os.makedirs('../output/visualizations', exist_ok=True)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Save processed data\n",
    "X_train_scaled.to_csv('../output/processed/X_train.csv', index=False)\n",
    "X_test_scaled.to_csv('../output/processed/X_test.csv', index=False)\n",
    "y_train.to_csv('../output/processed/y_train.csv', index=False)\n",
    "y_test.to_csv('../output/processed/y_test.csv', index=False)\n",
    "\n",
    "print(\"✅ Processed data saved to 'output/processed/' folder\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Save model coefficients\n",
    "coefficients.to_csv('../output/models/coefficients.csv', index=False)\n",
    "print(\"✅ Model coefficients saved to 'output/models/coefficients.csv'\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 10. 📋 Summary Report"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "print(\"\\n\" + \"=\"*60)\n",
    "print(\"📋 FINAL MODEL SUMMARY REPORT\")\n",
    "print(\"=\"*60)\n",
    "\n",
    "print(f\"\"\"\n",
    "1. DATASET INFORMATION\n",
    "   • Total samples: {len(X_train) + len(X_test)}\n",
    "   • Training samples: {len(X_train)} ({len(X_train)/(len(X_train)+len(X_test))*100:.1f}%)\n",
    "   • Testing samples: {len(X_test)} ({len(X_test)/(len(X_train)+len(X_test))*100:.1f}%)\n",
    "   • Features: {X_train.shape[1]}\n",
    "\n",
    "2. MODEL PERFORMANCE\n",
    "   {'─'*50}\n",
    "   {'Metric':<20} {'Training':>12} {'Test':>12}\n",
    "   {'─'*50}\n",
    "   {'R² Score':<20} {train_r2:>12.4f} {test_r2:>12.4f}\n",
    "   {'MSE':<20} {train_mse:>12.2f} {test_mse:>12.2f}\n",
    "   {'RMSE':<20} {train_rmse:>12.2f} {test_rmse:>12.2f}\n",
    "   {'MAE':<20} {train_mae:>12.2f} {test_mae:>12.2f}\n",
    "   {'─'*50}\n",
    "\n",
    "3. TOP 5 POSITIVE IMPACT FEATURES\n",
    "   {'─'*50}\"\"\")\n",
    "\n",
    "print(\"\\n   🔼 Features that INCREASE house price:\")\n",
    "for _, row in coefficients[coefficients['Coefficient'] > 0].head(5).iterrows():\n",
    "    print(f\"      • {row['Feature']:15}: +{row['Coefficient']:.4f}\")\n",
    "\n",
    "print(\"\\n   🔽 Features that DECREASE house price:\")\n",
    "for _, row in coefficients[coefficients['Coefficient'] < 0].head(5).iterrows():\n",
    "    print(f\"      • {row['Feature']:15}: {row['Coefficient']:.4f}\")\n",
    "\n",
    "print(f\"\"\"\n",
    "   {'─'*50}\n",
    "\n",
    "4. INTERPRETATION\n",
    "   • The model explains {test_r2*100:.1f}% of the variance in house prices.\n",
    "   • Average prediction error (MAE): ${test_mae:.2f}K.\n",
    "   • 95% of predictions are within ±${test_rmse*1.96:.2f}K.\n",
    "\"\"\")\n",
    "print(\"=\"*60)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 11. ✅ Conclusion"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Key Takeaways:\n",
    "\n",
    "1. **Model Performance**: The linear regression model explains **{test_r2*100:.1f}%** of the variance in house prices, which is quite good for a simple model.\n",
    "\n",
    "2. **Most Important Features**:\n",
    "   - **RM** (average number of rooms) - More rooms = higher price\n",
    "   - **LSTAT** (% lower status population) - Higher % = lower price\n",
    "   - **DIS** (distance to employment centers) - Further distance = higher price\n",
    "\n",
    "3. **Model Accuracy**: On average, predictions are off by **${test_mae:.2f}K**.\n",
    "\n",
    "4. **Next Steps**: To improve accuracy, we could try:\n",
    "   - Feature engineering (creating interaction terms)\n",
    "   - Polynomial features\n",
    "   - Regularization (Ridge/Lasso regression)\n",
    "   - Ensemble methods (Random Forest, Gradient Boosting)\n",
    "\n",
    "---\n",
    "\n",
    "**🎉 Task 2 Completed Successfully!**"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "name": "python",
   "version": "3.9.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}